In [ ]:
import os
from tqdm import tqdm

import pandas as pd
import numpy as np
import openpyxl as ox
import re

from openpyxl import load_workbook
from openpyxl import workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell.cell import WriteOnlyCell

from datetime import timedelta
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

import string

import pyodbc

#-------------------------------------------------------

def date(x):
    d = datetime(1900,1,1)
    y = timedelta(days=x-2)
    x = d+y
    x = x.strftime("%Y%m%d")
    return x


def product(x, y):
    x['ОПЛАТА ПО ФАКТУ'] = pd.to_datetime(x['ОПЛАТА ПО ФАКТУ'])
    x['Год_оплаты_факт'] = x['ОПЛАТА ПО ФАКТУ'].dt.year
    x['Месяц'] = x['ОПЛАТА ПО ФАКТУ'].dt.month
    x = x.query('Год_оплаты_факт == 2023')
    x['ID сделки'] = x['ID сделки'].drop_duplicates()
    x = x.loc[x['Продукт_2'] == y]
    x = x[['ID сделки', 'ОТДЕЛ', 'Продукт', 'ID лида', 'Клиент', 'Месяц']]
    x['Продукт'] = x['Продукт'].replace('курсы', 1)
    x = x.rename(columns={'Продукт': y})
#    x['ID сделки'] = x['ID сделки'].fillna(0)
#    x['ID сделки'] = x['ID сделки'].astype(int)
    return x

#-------------------------------------------------------

data = pd.read_excel(r'\\synergy.local\Documents\19.Группа мониторинга и сопровождения сделок\01.Отчеты\База ЧВ_бд18 0,25.xlsx', sheet_name='База ЧВ')

for row in tqdm(data['ОПЛАТА ПО ФАКТУ']):
    data.loc[data['ОПЛАТА ПО ФАКТУ'] == row, ['ОПЛАТА ПО ФАКТУ']] = date(row)
pass

sch = data
rp = data
two_in_one = data

#-------------------------------------------------------

conn = pyodbc.connect(r'Driver={SQL Server};Server=MSK1-BIDB01;Database=DWH;Trusted_Connection=yes;')
cursor = conn.cursor()

ls = f'''

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED

declare @date date = getdate()-1
declare @month int = month(@date)
declare @startdate date = DATEFROMPARTS(2022,01,01);

SELECT DISTINCT
     --даты
     D.CODE as "id deal crm"
     
      ,D.ID_CFO_VISUAL  as "Визуал"
     
       


FROM [DWH].[dbo].DIC_DEAL D
    left join [DWH].[dbo].[DIC_DEAL_VISUAL] V ON V.[ID_DEAL]=D.ID_DEAL
    LEFT JOIN [DWH].[dbo].[DIC_EMPLOYEES] ed ON ed.ID_EMPLOYEES = V.[ID_EMPLOYEES_VISUAL]

    LEFT JOIN [DWH].[dbo].[DIC_EMPLOYEES] E ON D.ID_EMPLOYEES = E.ID_EMPLOYEES
    LEFT JOIN [DWH].[dbo].[ASS_EMPLOYEE_AND_ORGSTRUCTURE] ASS_OS ON E.ID_EMPLOYEES = ASS_OS.ID_EMPLOYEES
    LEFT JOIN [DWH].[dbo].[v_DIC_ORGSTRUCTURE] org on org.ID_ORGSTRUCTURE = ASS_OS.ID_ORGSTRUCTURE
    

    
    -- статус сделки
    left join [DWH].[dbo].[DIC_STATUS_DEAL] sd on sd.ID_STATUS_DEAL = d.ID_STATUS_DEAL


  
WHERE
  D.ID_DEAL <> - 1

    and D.[DATE_CREATE]  between @startdate and @date -- фильтр по дате завершения 
and org.full_NAME not like '%МАП%'
and org.full_NAME like '%Коммерческий департамент (КМВ)\%'
and D.ID_CFO_VISUAL > 0

  
'''


Employees = f'''

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED

SELECT [EMPLOYEES]
      ,[LOG]
      ,[SP]

      ,CASE

       WHEN KC like '%4%' THEN 'КЦ 4'
       WHEN KC like '%3%' THEN 'КЦ 3'
       WHEN KC like '%центр 3%' THEN 'КЦ 3'
       WHEN KC like '%продаж 18%' THEN 'ОП 18'
             
       ELSE KC
  END
  as "KC"

         ,CASE

       WHEN [KC] like '%4%' THEN SUBSTRING([ОП],CHARINDEX(' ',[ОП])+1,LEN([ОП])) + ' ВР'
       WHEN ([ОП] in ('ОП 9')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('ОП 2')) and ([GP] in ('.1')) THEN 'ОП 8'
       WHEN ([ОП] in ('ОП 10')) and ([GP] in ('.2')) THEN 'ОП 10.2'
       WHEN ([ОП] in ('ОП 5')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('3 ЯР')) and ([GP] in ('.1')) THEN '3.1 ЯР'
       WHEN ([KC] in ('Отдел прямых продаж'))  THEN 'ОПП'
           WHEN [KC] like '%продаж 18%'  THEN 'ОП 18'  
       ELSE [ОП]
  END
  as "ОП"
      
               ,[PHONE]
      ,[STATUS]
      ,[ID_EMPL]
      ,[ID_ORG]

  FROM [DWH].[dbo].[KHTS_EMPL]
  where 
  --KC like '%4%' 
  [SP] in ('КД')


'''

ls = pd.read_sql_query(ls, conn)
Employees = pd.read_sql_query(Employees, conn)

#-------------------------------------------------------

ls = ls.query('Визуал == 64627099')
ls['Визуал'] = ls['Визуал'].replace(64627099, 1)
ls.columns = ['ID сделки', 'Визуал']
ls['ID сделки'] = ls['ID сделки'].astype(int)

sch = product(sch, "СЧ")
rp = product(rp, "РП")
two_in_one = product(two_in_one, "2в1")
two_in_one['2в1'] = two_in_one['2в1'].replace(1, 2)

#display(sch.info())
#display(rp.info())
#display(two_in_one.info())

#-------------------------------------------------------

data['ОПЛАТА ПО ФАКТУ'] = pd.to_datetime(data['ОПЛАТА ПО ФАКТУ'])
data['Год_оплаты_факт'] = data['ОПЛАТА ПО ФАКТУ'].dt.year
data['Месяц'] = data['ОПЛАТА ПО ФАКТУ'].dt.month
data = data.query('Год_оплаты_факт == 2023')
data['ID сделки'] = data['ID сделки'].drop_duplicates()
data = data.query('Продукт_2 != "РП" & Продукт_2 != "сч" & Продукт_2 != "СЧ"')
data = data.query('Чистая >= 20000')
data['ID сделки'] = data['ID сделки'].fillna(0)
data['ID сделки'] = data['ID сделки'].astype(int)

data = data.merge(ls, how='left', on='ID сделки')
data = data.query('Визуал != 1')

data = data[["ID сделки", "КЦ", "ОТДЕЛ", "ОПЕРАТОР", "ОПЛАТА ПО ФАКТУ", 
             "ДАТА ПОДАЧИ", "Чистая", "Продукт", "Срок", "Форма", "Клиент", 
             "Примечание", "Год_оплаты_факт", "Продукт_2", "Месяц", "Визуал"]]

#-------------------------------------------------------

wb = load_workbook(r'\\synergy.local\Documents\19.Группа мониторинга и сопровождения сделок\01.Отчеты\Аналитика КМВ\ДОП.xlsx')

sheet = wb.get_sheet_by_name('Sheet_1')
sheet.delete_rows(1, sheet.max_row)

for r in dataframe_to_rows(data, index=False, header=True):
    sheet.append(r)
    
sheet = wb.get_sheet_by_name('2в1')
sheet.delete_rows(1, sheet.max_row)

for r in dataframe_to_rows(two_in_one, index=False, header=True):
    sheet.append(r)

sheet = wb.get_sheet_by_name('РП')
sheet.delete_rows(1, sheet.max_row)

for r in dataframe_to_rows(rp, index=False, header=True):
    sheet.append(r)

sheet = wb.get_sheet_by_name('СЧ')
sheet.delete_rows(1, sheet.max_row)

for r in dataframe_to_rows(sch, index=False, header=True):
    sheet.append(r)



wb.save(r'\\synergy.local\Documents\19.Группа мониторинга и сопровождения сделок\01.Отчеты\Аналитика КМВ\ДОП.xlsx')

#-------------------------------------------------------

print("")
print("Готово!")

 65%|████████████████████████████████████████████████▊                          | 27094/41637 [01:41<00:45, 316.23it/s]